In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn as sk
from bertopic import BERTopic
import nltk
from gensim.models import Word2Vec

nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /Users/amanshah/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [5]:
data_raw = pd.read_excel("rosetta_data_inputs_2023.xls")
display(data_raw)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,RowGUID,ProjectID,Procestrin,IndexNo,Overskrift,Beskrivelse,Billedfil,Attribut1,Attribut2,Attribut3,...,Attribut11,Attribut12,Attribut13,Attribut14,Attribut15,Attribut16,Attribut17,Attribut18,Attribut19,Attribut20
0,8,1000,4,NaN,NaN,NaN,NaN,NaN,1207,NaN,...,NaN,NaN,NaN,NaN,NaN,1207,NaN,NaN,NaN,1207
1,43,39,1,NaN,Combi-material-design,"Use a combination of materials, such as brick,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,45,39,1,NaN,Sterilization with UV light,UVC light can be used to quickly disinfect and...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46,39,1,NaN,seperation by oscillation,Protein powder is introduced to a plug flow tu...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,47,37,1,NaN,high pressure,expose the product to high enough pressure to ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5487,6006,42,3,NaN,Weather resistance inspired by arctic animals,Inspired by animals who adapted their body sha...,NaN,NaN,202210066,Stability: By designing the structure with aer...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5488,6007,42,3,NaN,Multiple options for power generation,"To provide power for the sensory equipment, th...",NaN,NaN,201906635,Versatility: The option to select the desired ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5489,6008,42,3,NaN,Adaptable,To increase the maneuverability of the structu...,NaN,NaN,202210066,Versatility: An adaptable structure can be use...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5490,6009,42,3,NaN,Weather balloon for communication relay,Satellite communication may not be an option i...,NaN,NaN,201709746,Weather balloons are much cheaper and easier t...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Get main columns and filter rows without headings
data = data_raw[["Overskrift", "Beskrivelse"]]
data = data[data['Overskrift'].notna()]

In [6]:
# remove non-english words

def only_english(text):
       return " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())

In [8]:
# prep data for use with BERTopic

# combine heading and description
title_strings = data['Overskrift'].astype('str')
description_strings = data['Beskrivelse'].astype('str')
docs = title_strings + " \n\n" + description_strings

docs.apply(only_english)

1       - material - design Use a combination of , suc...
2       Sterilization with light light can be used to ...
3       by oscillation Protein powder is to a plug flo...
4       high pressure expose the product to high enoug...
5       Filtration push the product through a membrane...
                              ...                        
5487    Weather resistance inspired by arctic Inspired...
5488    Multiple for power generation To provide power...
5489    Adaptable To increase the maneuverability of t...
5490    Weather balloon for communication relay Satell...
5491    Adaptable sensor image frequency and resolutio...
Length: 5339, dtype: object

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# Remove stopwords after documents are assigned to topics
# Allows the transformer model to get the full context of the 
# data, while removing stopwords that are noise in the topics
vectorizer_model = CountVectorizer(stop_words="english")

# Define and train the model
topic_model = BERTopic(nr_topics=20, vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(docs)

KeyboardInterrupt: 

In [35]:
topic_model.get_topic_info()  

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1051,-1_nan_use_using_water,"[nan, use, using, water, robot, surface, air, ...",[Using high temperature boling oils as an heat...
1,0,973,0_nan_blinds_robot_shot,"[nan, blinds, robot, shot, ground, explosives,...",[A blind were you can see out but not in \n\nn...
2,1,743,1_cups_cup_people_board,"[cups, cup, people, board, glands, plastic, pa...","[A library where your cups are stored, then th..."
3,2,488,2_tank_heat_energy_exchanger,"[tank, heat, energy, exchanger, pumice, tanks,...",[Hybrid engines \n\nHybrid engines could be us...
4,3,470,3_weeds_toilet_detect_use,"[weeds, toilet, detect, use, plants, urine, sm...",[In car cleaning gas chamber \n\nTargeting the...
5,4,207,4_oven_pizza_buns_bread,"[oven, pizza, buns, bread, rolls, random, diff...",[Big pizza oven \n\nMake the pizza oven bigger...
6,5,202,5_fish_chickens_feed_dead,"[fish, chickens, feed, dead, water, chicken, f...",[Automatically feed the fish when needed \n\nU...
7,6,177,6_berries_pulp_potatoes_starch,"[berries, pulp, potatoes, starch, potato, cutt...",[Cutting the berries as thin as possible \n\nn...
8,7,167,7_window_windows_glass_old,"[window, windows, glass, old, frame, frames, g...",[Make different sizes of the old window \n\nVe...
9,8,162,8_chair_wheels_table_elderly,"[chair, wheels, table, elderly, chairs, citize...",[Assumption Future (No care workers) - Fully a...


In [13]:
# set up Word2Vec embeddings

w2v = Word2Vec(docs, window=5, workers=4, min_count=3)

In [19]:
# do PCA

from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_result = pca.fit_transform(w2v.wv.vectors)

In [21]:
import seaborn as sns

sns.scatterplot(data=pca_result, x=0, y=1)
plt.title("Transformed data")
plt.xlabel("First Eigenvector")
plt.ylabel("Second Eigenvector")
plt.show()

TypeError: Data source must be a DataFrame or Mapping, not <class 'numpy.ndarray'>.